In [2]:
from azure.identity import AzureCliCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, ChatCompletionAgent
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FunctionTool, ToolSet
from dotenv import load_dotenv
import asyncio
import requests
from semantic_kernel.functions import kernel_function
import pandas as pd
load_dotenv()

model = os.getenv("AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME")

project_client = AzureAIAgent.create_client(credential=AzureCliCredential(),
                           conn_str=os.getenv("PROJECT_CONNECTION_STRING")
)



class RecognizedRevenue:
    @kernel_function(description="Gets recognized revenue, invoice date, billing date, and due date for single or multiple companies.")
    def get_recognized_revenue(self, companyName: str):
        # Load CSV file
        df = pd.read_csv("recognizedRevenue.csv")
        companyName = companyName.strip().lower()

        if companyName == "all":
            # Return full records for all companies
            return df.to_dict(orient="records")
        
        # Match specific company
        matched_row = df[df["Company"].str.lower() == companyName]
        
        if not matched_row.empty:
            return matched_row.to_dict(orient="records")[0]
        else:
            return f"Company '{companyName}' not found in the data."

# one-time-payment-percent.csv

class HistoricalPayment:

    @kernel_function(description="Gets historical payment 'on-time' patterns.")
    def get_historical_payment_patterns(self, companyName: str):
        try:
            df = pd.read_csv("one-time-payment-percent.csv")  # Update path if needed
            df["Company"] = df["Company"].str.strip().str.lower()
            companyName = companyName.strip().lower()

            if companyName == "all":
                return dict(zip(df["Company"].str.title(), df["On-time Payment Percent"]))
            
            # Find specific company
            match = df[df["Company"] == companyName]
            if not match.empty:
                return str(match.iloc[0]["On-time Payment Percent"]) + "%"
            else:
                return f"Company '{companyName}' not found in historical data."

        except Exception as e:
            return f"Error reading historical payments: {str(e)}"

class CashFlow:

    @kernel_function(description="Calculates cash flow.")
    def cash_flow_calculation(self, inflow:int, outflow:int, riskyCustomerRevenue:int):
        return inflow-outflow-riskyCustomerRevenue

class ShipmentInformation:

    @kernel_function(description="Checks if the stocks are available for the number of days passed.")
    def get_shipment_information(self, noOfDays:int):
        if noOfDays>=60:
            return f"Stocks for {noOfDays} is not available."
        else:
            return f"Stocks for {noOfDays} is available."


class InventoryInformation:

    @kernel_function(description="Gets inventory stock size.")
    def get_inventory_information(self):
        return "50000"

        
data_access_agent_definition = await project_client.agents.create_agent(
        model=model,
        name="Data_Access_Agent",
        instructions="""
        You are a specialized AI agent responsible for providing access to financial data upon request
        by other agents in the system. You hold complete and accurate data for on-time payment history, revenue recognition,
        cash flow calculation, shipment information and inventory information. 
        Other agents depend on you to retrieve or analyze data. If recognized revenue is required, call 'RecognizedRevenue()'
        , if historical payment 'on-time' pattern is required, call 'HistoricalPayment()', if inventory stock related enquiry
        comes, call 'InventoryInformation()' and if enquiry comes to Checks if the stocks are available for certain 
        number of days then call 'ShipmentInformation()'.

        If recognized revenue data (including invoice, due, and payment dates) is required for one or more companies, 
        call 'RecognizedRevenue()'. If the user says "all", return data for all companies. Otherwise, return data for that 
        specific company.

        """,
    )

DataAccessAgent = AzureAIAgent(
        client=project_client,
        definition=data_access_agent_definition,
        plugins=[RecognizedRevenue(), HistoricalPayment(), ShipmentInformation(), InventoryInformation()]
    )


In [3]:

from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

service = AzureChatCompletion(
    api_key=os.getenv("AZURE_API_KEY"),
    endpoint=os.getenv("AZURE_ENDPOINT"),
    api_version=os.getenv("API_VERSION"),
    deployment_name=os.getenv("AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME"),
)

Supply_Chain_Agent = ChatCompletionAgent(
    service=service,
    name="SupplyChainAgent",
    instructions=(
        """
        You are a specialized SupplyChain Agent responsible for handling all requests related to inventory 
        status and shipment information across the system. Any time inventory or shipping data is requested, you are 
        the designated agent to process and fulfill that request. You call a DataAccessAgent for accessing data.  And 
        also instruct the DataAccess Agent to fetch stock or inventory data as per requirement.
        """

    ),
    plugins=[DataAccessAgent]
)


In [4]:

Finance_Agent = ChatCompletionAgent(
    service=service,
    name="FinanceAgent",
    instructions=(
        """
        You are the Finance Agent. Your responsibility is to respond to cash inflow and outflow related queries,
        as well as queries about recognized revenue details.

        For revenue and invoice-related data (including invoice date, due date, and payment date), call the DataAccessAgent
        using 'RecognizedRevenue()'.

        - If the user asks for data about all companies, pass 'all' as the company name and return the full table.
        - If the user asks for a specific company, return only that company's data.

        Note that the total cash outflow is $90,000. If you identify any payment delays or discrepancies, send a message to the FPA Agent.
        
        """
    ),
    plugins=[DataAccessAgent]
)

In [5]:
Sales_AM_Agent = ChatCompletionAgent(
    service=service,
    name="SalesAndAMAgent",
    instructions=(
        """
        You are the Sales and AM Agent responsible for managing customer payment terms and contracts.
        Your goal is to propose or respond to negotiated terms in a way that minimizes financial risk while maintaining good customer relationships.

        Use DataAccessAgent to retrieve:
        - Call DataAccess agent to callm'HistoricalPayment(companyName) for speicific or 'HistoricalPayment(all)' 
        for insights on their behavior.

        Based on the payment risk, agree on appropriate terms:
        - For high-risk customers, consider Net 45 terms with a '2%' early payment discount and '1%' late penalty clause.
        - Confirm accepted terms and update them in the system using the DataAccessAgent.

        Always communicate final terms back to the FP&A Agent.
        """
    ),
    plugins=[DataAccessAgent]
)


In [6]:
FPA_Agent = ChatCompletionAgent(
    service=service,
    name="FPA_Agent",
    instructions=(
        """
        You are a FPA Agent (Orchestrator agent). for recognized revenue call ask Data access agent and ask it to call 
        RecognizedRevenue() , and forecast cash flow based on 
        historical payments pattern, inventory data (Inventory Data from Supply Chain Agent) from DataAccess Agent. 
        If any data is required, call DataAccess Agent and provide instruction to it like what kind of data is needed.
        If any company has on-time payment less than '70%' then print 'RiskyCustomer : customer_name'. 
        If cash flow calculation is needed then call 'CashFlow()' (If there is no risky customer 
        send 0 for riskyCustomerRevenue parameter).

        You communicate with Sales AM agent to agree upon stricter terms for risky customer to reduce their due date.
        agree upon terms For high-risk customers, agree upon the plan intimated my Sales AM agent. 
        Do not Reduce it yourself.

        You call Finance Agent to retrieve revenue and invoice-related data (including invoice date, due date, and payment date)
        , call the Finance agent and fetch data using DataAccessAgent using 'RecognizedRevenue()'.

        Also Print the name of the agent along with their response when You are calling in each step in [].

        """

    ),
    plugins=[DataAccessAgent, CashFlow(),Supply_Chain_Agent, Sales_AM_Agent, Finance_Agent]
)


In [7]:

# user_input = input()

# thread = project_client.agents.create_thread()


# async def get_response_from_agent():
#     response =  await FPA_Agent.get_response(
#         messages = user_input,
#     )
    
#     return response
   
# response = await get_response_from_agent()
# print(response)



In [8]:

from semantic_kernel.agents import ChatHistoryAgentThread

async def chat(FPA_Agent: ChatCompletionAgent, thread: ChatHistoryAgentThread = None) -> bool:
    """
    Continuously prompt the user for input and show the assistant's response.
    Type 'exit' to exit.
    """
    try:
        user_input = input("User:> ")
    except (KeyboardInterrupt, EOFError):
        print("\n\nExiting chat...")
        return False

    if user_input.lower().strip() == "exit":
        print("\n\nExiting chat...")
        return False

    response = await FPA_Agent.get_response(
        messages=user_input,
        thread=thread,
    )

    if response:
        print(f"Agent :> {response}")

    return True


thread = project_client.agents.create_thread()
thread: ChatHistoryAgentThread = None

print("Welcome to the chat bot!\n  Type 'exit' to exit.")

chatting = True
while chatting:
    chatting = await chat(FPA_Agent, thread) 

C:\Users\Avik\AppData\Local\Temp\ipykernel_73772\3679904902.py:30: RuntimeWarning: coroutine 'AgentsOperations.create_thread' was never awaited
  thread: ChatHistoryAgentThread = None


Welcome to the chat bot!
  Type 'exit' to exit.
Agent :> The total cash flow calculated for the next 30 days is **$1,138,326**. 

### Risky Customers Identified:
1. **Dell**: On-time payment rate of 62%.
2. **Tesla**: On-time payment rate of 68%.

### Revised Payment Plan Suggested:
For **Dell** and **Tesla**, the recommended terms are as follows:
- **Net 45 Payment Terms**: Payment to be made within 45 days.
- **2% Early Payment Discount**: Incentive for payments made early.
- **1% Late Penalty Clause**: To address late payments.

It is advised to coordinate with the legal or finance team to formally communicate and update these terms in the system for these customers. 

Let me know if you need further assistance!


Exiting chat...


In [9]:
# thread = project_client.agents.create_thread()
# thread: ChatHistoryAgentThread = None

# print("Welcome to the chat bot!\n  Type 'exit' to exit.\n")

# chatting = True
# while chatting:
#     chatting = await chat(FPA_Agent, thread)

In [15]:
from semantic_kernel.agents import ChatHistoryAgentThread

async def chat(agent: ChatCompletionAgent, thread: ChatHistoryAgentThread = None) -> bool:
    """
    Continuously prompt the user for input and show the agent's response.
    Type 'exit' to exit.
    """
    try:
        user_input = input("User:> ")
    except (KeyboardInterrupt, EOFError):
        print("\n\nExiting chat...")
        return False

    if user_input.lower().strip() == "exit":
        print("\n\nExiting chat...")
        return False

    response = await agent.get_response(
        messages=user_input,
        thread=thread,
    )

    if response:
        print(f"[{agent.name}]: {response}")  # <-- Print agent name with response

    return True


In [16]:
agents = [FPA_Agent, Supply_Chain_Agent, Sales_AM_Agent,Finance_Agent]  # Add others

async def orchestrate_agents(user_input: str, thread=None):
    for agent in agents:
        response = await agent.get_response(messages=user_input, thread=thread)
        print(f"[{agent.name}]: {response}")

In [17]:
while True:
    user_input = input("User:> ")
    if user_input.lower().strip() == "exit":
        print("\nExiting chat...")
        break
    await orchestrate_agents(user_input, thread)


[FPA_Agent]: The cash flow for the next 30 days has been calculated as **$1,140,326**.

### Key Details:
- **Total Inflow**: $1,624,313 (derived from recognized revenue)
- **Total Outflow**: $90,000
- **Risky Customer Revenue**: $393,987 (Dell and Tesla identified as high-risk customers)

### Actions Taken:
1. **Strict Terms**: Adjusted terms for Dell and Tesla with:
   - Net 45 payment terms
   - 2% early payment discount
   - 1% late penalty clause
2. **Risk Identification**: "RiskyCustomers" identified:
   - Dell
   - Tesla
3. **Inventory Checked**: No logistical constraints detected for shipments, and the current stock is 50,000 units.

Let me know if you need further assistance!
[SupplyChainAgent]: I am specialized in handling inventory and shipment-related requests only, and I am not equipped to handle financial calculations like cash flow.

For cash flow data, you would need to contact a Finance or Accounting-specific Agent. Let me know if you need assistance with inventory or s